In [448]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import tensorflow as tf
import pandas as pd

In [449]:
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = False
configuration.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.compat.v1.Session(config=configuration)


In [450]:
data=pd.read_csv('datasets/malicious_data_generated.csv')
npdata=data.to_numpy()
malData=np.copy(npdata)
print(type(malData[1,:]))
print(malData[:1].shape)

<class 'numpy.ndarray'>
(1, 10)


In [451]:
class MalwareEnv():
    def __init__(self):
        # Actions we can take, decrease, increse, none
        self.action_space = Discrete(20)
        # max-min array
        self.observation_space = Box(low=-100,high=100,shape=(10,))
        
        # Set start 
        self.state = np.asarray(malData[random.randint(0,499),:])
        
        # Set time 
        self.length = 60
        
    def step(self, action):
        # Apply action for each state
        if(action<10):
            if(action==0):
                self.state[0]=self.state[0]+1
            elif(action==1):
                 self.state[1]=self.state[1]+1
            elif(action==2):
                 self.state[2]=self.state[2]+1
            elif(action==3):
                 self.state[3]=self.state[3]+1
            elif(action==4):
                 self.state[4]=self.state[4]+1
            elif(action==5):
                 self.state[5]=self.state[5]+1
            elif(action==6):
                 self.state[6]=self.state[6]+1
            elif(action==7):
                 self.state[7]=self.state[7]+1
            elif(action==8):
                 self.state[8]=self.state[8]+1
            else:
                 self.state[9]=self.state[9]+1                     
        else:
            if(action==10):
                self.state[0]+=self.state[0]-1
            elif(action==11):
                 self.state[1]+=self.state[1]-1
            elif(action==12):
                 self.state[2]+=self.state[2]-1
            elif(action==13):
                 self.state[3]+=self.state[3]-1
            elif(action==14):
                 self.state[4]+=self.state[4]-1
            elif(action==15):
                 self.state[5]+=self.state[5]-1
            elif(action==16):
                 self.state[6]+=self.state[6]-1
            elif(action==17):
                 self.state[7]+=self.state[7]-1
            elif(action==18):
                 self.state[8]+=self.state[8]-1
            else:
                 self.state[9]+=self.state[9]-1  
        
        self.length -= 1 
            
        
        # Calculate reward in ranges
        rewardMulti=0 
        
        if(self.state[0]>=-0.290698 and self.state[0]<=-133.441860):
                 rewardMulti=rewardMulti+1
        elif(self.state[1]>=0 and self.state[1]<=1184):
                 rewardMulti=rewardMulti+1
        elif(self.state[2]>=-0.666667 and self.state[2]<=10.666667):
                 rewardMulti=rewardMulti+1
        elif(self.state[3]>=-0.312383 and self.state[3]<=109.259173):
                 rewardMulti=rewardMulti+1
        elif(self.state[4]>=0 and self.state[4]<=30):
                 rewardMulti=rewardMulti+1
        elif(self.state[5]>=-0.322 and self.state[5]<=127.488889):
                 rewardMulti=rewardMulti+1
        elif(self.state[6]>=-0.282353 and self.state[6]<=147.976471):
                 rewardMulti=rewardMulti+1
        elif(self.state[7]>=-0.164688 and self.state[7]<=715.616633):
                 rewardMulti=rewardMulti+1
        elif(self.state[8]>=-0.324081 and self.state[8]<=106.407677):
                 rewardMulti=rewardMulti+1
        elif(self.state[9]>=-0.750000 and self.state[9]<=227.5):
                 rewardMulti=rewardMulti+1
                 
        if(rewardMulti==0):
            reward=-1
        else:
            reward=rewardMulti
                 
        
        # Check if is done
        if self.length <= 0: 
            done = True
        else:
            done = False
        
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        pass
    
    def reset(self):
        # Reset 
        self.state = malData[random.randint(0,498),:]
        # Reset time
        self.length = 60 
        return self.state

In [452]:
env = MalwareEnv()

In [453]:
env.observation_space.sample()

array([-51.34826 , -57.779793, -60.866684,  59.133427,  76.72466 ,
       -17.644281, -52.234383,  70.68254 , -30.749245,  90.930695],
      dtype=float32)

In [454]:
with tf.device('/gpu:0'):
    episodes = 10
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 
    
        while not done:
            #env.render()
            action = env.action_space.sample()
            n_state, reward, done, info = env.step(action)
            score+=reward
        print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:60
Episode:2 Score:60
Episode:3 Score:60
Episode:4 Score:60
Episode:5 Score:60
Episode:6 Score:60
Episode:7 Score:60
Episode:8 Score:28
Episode:9 Score:60
Episode:10 Score:60


In [455]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow

In [456]:
states = env.observation_space.shape
actions = env.action_space.n

In [457]:
actions

20

In [458]:
states

(10,)

In [459]:
def build_model(states, actions):
    with tf.device('/gpu:0'):
        model = tensorflow.keras.models.Sequential()   
        model.add(tensorflow.keras.layers.Dense(24, activation='relu', input_shape=(1,10)))
        model.add(tensorflow.keras.layers.Dense(24, activation='relu'))
        model.add(tensorflow.keras.layers.Dense(20, activation='linear'))
        model.add(Flatten())

        return model

In [460]:

#with tf.device('/gpu:0'):
model = build_model(states, actions)

In [461]:
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 1, 24)             264       
_________________________________________________________________
dense_92 (Dense)             (None, 1, 24)             600       
_________________________________________________________________
dense_93 (Dense)             (None, 1, 20)             500       
_________________________________________________________________
flatten_3 (Flatten)          (None, 20)                0         
Total params: 1,364
Trainable params: 1,364
Non-trainable params: 0
_________________________________________________________________


In [462]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [463]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [464]:
with tf.device('/gpu:0'):
    dqn = build_agent(model, actions)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 184s 18ms/step - reward: 0.9854
done, took 184.206 seconds
